### Introduction à l'IA générative avec l'API OpenAI

## Plan global du Notebook

1. **Introduction Générale**  
   - Contexte et définitions (IA Générative, LLMs, prompts, etc.)
   - Présentation succincte des enjeux éthiques et de responsabilité

2. **Premier Exemple de Code : Vérification de l'Environnement**  
   - Installation et importation des bibliothèques (OpenAI ou Azure OpenAI)
   - Test d’un prompt de base

3. **Notions de Base sur les Prompts**  
   - Tokens, embeddings, bases de la génération
   - Exercices pratiques : tokenisation et génération

4. **Exemple : Fabrications (Hallucinations) et Fiabilité**  
   - Démonstration via un prompt volontairement ambigu
   - Discussion sur la vérification des faits

5. **Conclusion**  
   - Récapitulatif des points abordés
   - Proposition d’activité (rédaction d’une synthèse ou d’une extension)

---

# Introduction à l'IA Générative

Dans ce notebook, nous explorerons les fondamentaux de l’Intelligence Artificielle Générative :

**Objectifs :**
- Comprendre ce qu’est l’IA Générative (texte, images, audio…)
- Découvrir le fonctionnement des modèles de langage de grande taille (LLMs)
- Mettre en pratique des expérimentations simples avec les prompts
- Aborder les questions de fiabilité (hallucinations) et d’éthique

## Qu’est-ce que l’IA Générative ?

L’IA générative est une branche de l’apprentissage automatique qui **génère** de nouveaux contenus (texte, image, audio, code…) à partir de modèles probabilistes entraînés sur de vastes ensembles de données. Les modèles les plus avancés, appelés **Large Language Models** (LLMs), utilisent des architectures de type **Transformer** pour prédire et générer des séquences.

Exemples concrets :
- **ChatGPT** (OpenAI) : génération et compréhension de texte
- **Stable Diffusion** : génération d’images
- **Audiocraft** et **Whisper** : génération et transcription audio
- **Copilot** (GitHub) : génération de code et assistance à la programmation

## Enjeux et Limites

- **Hallucinations / Fabrications** : Le modèle peut générer des réponses inventées ou inexactes, même si elles semblent plausibles.
- **Biais** : Les réponses peuvent refléter des biais présents dans les données d’entraînement.
- **Coût Énergétique** : L’entraînement de grands modèles consomme beaucoup de ressources.
- **Régulation et Éthique** : Confidentialité, respect des lois et usage responsable de la technologie.

Nous allons maintenant configurer l'environnement et réaliser un premier test de prompt.


In [1]:
# ==========================================
# Cellule 2 : Installation et Configuration
# ==========================================

# Installation des packages nécessaires (à lancer uniquement si non déjà installés)
%pip install openai tiktoken python-dotenv

import os
from openai import OpenAI
from dotenv import load_dotenv

# Charger la configuration depuis le fichier .env (dans le répertoire parent GenAI/)
load_dotenv('../.env')

# Initialiser le client OpenAI (détecte automatiquement OPENAI_API_KEY)
client = OpenAI()

print("Client OpenAI initialisé avec succès (syntaxe moderne) !")

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Client OpenAI initialisé avec succès (syntaxe moderne) !


### Qu'est-ce qu'un prompt ?

Dans le contexte des LLMs (Large Language Models), un **prompt** est la consigne initiale que l’on fournit au modèle. Il peut s'agir d'une question, d'une instruction, d'un texte partiel, voire d'un exemple de conversation. Le prompt influe directement sur la qualité de la réponse générée.

### Chat Completions vs. Completions

OpenAI propose principalement deux approches pour générer du texte :

1. **Completions API** (versions historiques)  
   - On envoie un simple prompt (ex.: `text-davinci-003`) et on récupère un texte.  
   - Peu pratique pour les dialogues complexes, car il faut manuellement gérer l’historique de la conversation.

2. **Chat Completions API** (recommandée)  
   - On structure le prompt en plusieurs messages avec des rôles (`system`, `user`, `assistant`, etc.).  
   - Permet des conversations plus riches (mémoire de conversation, enchaînements de tours) et un meilleur contrôle du style.  

Dans ce notebook, nous utilisons principalement la **Chat Completions API** via `client.chat.completions.create()` (syntaxe moderne). 
[En savoir plus dans la documentation officielle](https://platform.openai.com/docs/api-reference/chat).

---


### Premier prompt

La list des paramètres est accessible dans le [documentation officielle](https://platform.openai.com/docs/api-reference/chat/create).

In [ ]:
# ============================
# Cellule 3 : Premier Prompt
# ============================

# Exemple simple : Génération d'une courte phrase à partir d'un prompt de base
# Utilisation de l'API Chat avec le modèle gpt-5-mini

response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Allons enfants de la Patrie, "
        }
    ],
    model="gpt-5-mini"
)

print("Réponse du modèle :")
print(response.choices[0].message.content)


### Interprétation du résultat

Le modèle a complété le prompt « Allons enfants de la Patrie, » en continuant naturellement les paroles de **La Marseillaise**, l'hymne national français. Cela démontre plusieurs capacités importantes des LLMs :

1. **Mémoire culturelle** : Le modèle a été entraîné sur de vastes corpus de textes et reconnaît des références culturelles célèbres.

2. **Prédiction contextuelle** : À partir d'un début de phrase, le modèle génère la suite la plus probable statistiquement.

3. **Génération naturelle** : Le modèle continue le texte de manière cohérente sans paramètres de limitation explicites.

**Paramètres utilisés** :
- `model="gpt-5-mini"` : Version économique et rapide du modèle GPT-5
- Pas de limitation de tokens : Le modèle génère jusqu'à ce qu'il considère la réponse complète

### Analyse de l'impact du message 'system'

Cette cellule illustre l'importance du **rôle 'system'** dans les conversations avec les LLMs :

**Rôle des messages** :
- **system** : Définit le comportement global et le style du modèle (instructions méta)
- **user** : Les questions ou demandes de l'utilisateur
- **assistant** : Les réponses précédentes du modèle (dans un contexte conversationnel)

Dans cet exemple, le message system « Tu es un assistant poétique qui répond toujours en haiku » transforme complètement la nature de la réponse. Au lieu d'une réponse explicative classique, le modèle produit un poème court de 3 vers (5-7-5 syllabes).

**Applications pratiques** :
- Chatbots avec personnalité spécifique (formel, humoristique, technique...)
- Assistants spécialisés (code, marketing, éducation...)
- Contrôle du ton et du format de sortie

**Note** : `temperature=0.7` permet une certaine créativité tout en restant cohérent.

In [ ]:
# ============================
# Cellule 4 (NOUVELLE) : Prompt avec un message 'system'
# ============================

# Ici, on utilise un message "system" pour donner une consigne globale sur le style du modèle.
# Le message "user" reste notre question.

messages = [
    {
        "role": "system",
        "content": "Tu es un assistant poétique qui répond toujours en haiku. "
    },
    {
        "role": "user",
        "content": "Que penses-tu de la tour Eiffel ?"
    },
    {
        "role": "assistant",
        "content": "Fleur de fer dressée\nParis tisse son ciel d'acier\nÉtreint les nuages doux"
    },
    {
        "role": "user",
        "content": "Que penses-tu de ton poème ?"
    }
]

response_system = client.chat.completions.create(
    messages=messages,
    model="gpt-5-mini"
)

print("=== Réponse avec un message 'system' ===")
print(response_system.choices[0].message.content)


### Analyse comparative de la tokenisation

Les résultats ci-dessus révèlent des **différences importantes** entre les tokenizers :

**Observations clés** :
1. **Nombre de tokens différent** : La même phrase peut être découpée en un nombre différent de tokens selon le modèle
2. **Granularité variable** : Certains tokenizers créent des tokens plus fins (ex: espaces séparés), d'autres plus larges
3. **Traitement des espaces** : Notez comment les espaces sont parfois inclus dans les tokens, parfois séparés

**Implications pratiques** :
- **Facturation** : Chaque appel API est facturé selon le nombre de tokens (input + output)
- **Limites de contexte** : Chaque modèle a une fenêtre maximale (ex: 8k, 32k, 128k tokens)
- **Optimisation** : Pour un même texte, le coût peut varier selon le modèle choisi

**Exemple concret** :
Si `text-davinci-003` utilise 5 tokens et `gpt-4o` en utilise 4 pour la même phrase, cela représente une économie de 20% sur le volume de tokens à traiter.

**Recommandation** : Toujours vérifier la tokenisation avant d'envoyer de longs documents pour estimer le coût réel.

# Notions de Base : Tokenisation

Lorsque nous envoyons un prompt à un LLM, le texte est converti en une séquence de **tokens**.  
- Un **token** est généralement un morceau de mot, un caractère spécial ou un sous-mot.  
- Chaque modèle possède sa propre manière de découper le texte, influençant le nombre total de tokens.  

**Pourquoi c’est important ?**  
- La facturation (ou la limitation) se base souvent sur le nombre total de tokens (entrée + sortie).  
- Une requête trop longue peut dépasser la *context window* du modèle (limite de tokens cumulés).  

> **Bonnes pratiques**  
> - Surveiller la longueur du prompt pour limiter le coût et éviter les dépassements.  
> - Utiliser des fonctions d’analyse (ex. `tiktoken`) pour **estimer** le nombre de tokens d’un texte avant l’envoi.  
> - Tester divers modèles (`text-davinci-003`, `gpt-4`, `gpt-4o-mini`, etc.) car la tokenisation et le coût peuvent varier.

Dans la bibliothèque `tiktoken` d'OpenAI, on peut directement encoder et décoder les tokens pour comprendre la segmentation.  
Ci-dessous, un exemple détaillé de la façon dont la phrase « Oh say can you see » est découpée différemment selon le modèle.


In [7]:
# ============================
# Cellule 5 : Analyse de la Tokenisation
# ============================

import tiktoken

# --- Partie 1 : Utilisation de l'encodeur pour "text-davinci-003" ---
encoder_td = tiktoken.encoding_for_model("text-davinci-003")
texte = "Oh say can you see"
tokens_td = encoder_td.encode(texte)
print("Liste des tokens pour text-davinci-003 (indexes) :\n", tokens_td)

# Décodage token par token
decoded_td = [encoder_td.decode([t]) for t in tokens_td]
print("\nDécodage token par token (text-davinci-003) :")
for i, token in enumerate(decoded_td):
    print(f"Token {i}: '{token}'")

# --- Partie 2 : Utilisation de l'encodeur pour "gpt-4o" (ou gpt-4o-mini) ---
encoder_gpt4 = tiktoken.encoding_for_model("gpt-4o")
tokens_gpt4 = encoder_gpt4.encode(texte)
print("\nListe des tokens pour gpt-4o (indexes) :\n", tokens_gpt4)

decoded_gpt4 = [encoder_gpt4.decode([t]) for t in tokens_gpt4]
print("\nDécodage token par token (gpt-4o) :")
for i, token in enumerate(decoded_gpt4):
    print(f"Token {i}: '{token}'")
    
# --- Partie 3 : Utilisation de l'encodeur pour "gpt-5" (ou gpt-5-mini) ---
encoder_gpt5 = tiktoken.encoding_for_model("gpt-5-mini")
tokens_gpt5 = encoder_gpt5.encode(texte)
print("\nListe des tokens pour gpt-5 (indexes) :\n", tokens_gpt5)

decoded_gpt5 = [encoder_gpt5.decode([t]) for t in tokens_gpt5]
print("\nDécodage token par token (gpt-5) :")
for i, token in enumerate(decoded_gpt5):
    print(f"Token {i}: '{token}'")


Liste des tokens pour text-davinci-003 (indexes) :
 [5812, 910, 460, 345, 766]

Décodage token par token (text-davinci-003) :
Token 0: 'Oh'
Token 1: ' say'
Token 2: ' can'
Token 3: ' you'
Token 4: ' see'

Liste des tokens pour gpt-4o (indexes) :
 [18009, 2891, 665, 481, 1921]

Décodage token par token (gpt-4o) :
Token 0: 'Oh'
Token 1: ' say'
Token 2: ' can'
Token 3: ' you'
Token 4: ' see'

Liste des tokens pour gpt-5 (indexes) :
 [18009, 2891, 665, 481, 1921]

Décodage token par token (gpt-5) :
Token 0: 'Oh'
Token 1: ' say'
Token 2: ' can'
Token 3: ' you'
Token 4: ' see'


### Analyse du phénomène d'hallucination

Le résultat ci-dessus est très instructif sur les **limites des LLMs** :

**Observation** : Le modèle a probablement généré une réponse plausible mais **totalement inventée**, car il n'y a eu aucune guerre sur Mars en 2076 (cet événement n'existe pas).

**Pourquoi le modèle invente-t-il ?**
1. **Architecture probabiliste** : Le modèle prédit les tokens les plus probables selon son entraînement, sans vérifier la véracité
2. **Pression à répondre** : Par défaut, le LLM tente de fournir une réponse même en l'absence d'information
3. **Cohérence narrative** : Le modèle génère des détails cohérents entre eux (noms de traités, dates, acteurs) qui renforcent l'illusion de véracité

**Comment réduire les hallucinations ?**
- **Prompt engineering** : « Si tu ne sais pas, dis "Je ne sais pas" »
- **RAG (Retrieval Augmented Generation)** : Fournir des sources documentaires vérifiées
- **Validation externe** : Vérifier les faits importants avec des sources fiables
- **Temperature basse** : Réduire la créativité pour des tâches factuelles

**Leçon importante** : Ne jamais faire confiance aveuglément aux réponses d'un LLM, surtout sur des faits historiques, médicaux, juridiques ou scientifiques. Toujours **vérifier les sources**.

## Exercice : Impact de la Température

Testez différentes valeurs de température pour constater l'impact sur la créativité des réponses.

1. Créez un nouveau prompt qui demande une courte histoire (par exemple : *“Raconte une histoire de 3 lignes sur un chat aventurier.”*).
2. Réglez la température à 0.0, exécutez la cellule et notez la réponse.
3. Réglez ensuite la température à 1.0 (ou même 1.2 si le modèle l’accepte), et comparez la différence de style ou de structure.

> **Remarques :**
> - Une température basse (~0.0) rend le modèle plus “strict”, proche d’une réponse déterministe.  
> - Une température haute (~1.0 ou plus) favorise la créativité mais peut entraîner des réponses moins cohérentes ou plus fantaisistes.


# Exemple de Fabrication (ou "Hallucination")

Les modèles de langage peuvent parfois générer des informations inventées ou inexactes.  
Pour illustrer ce phénomène, nous allons utiliser un prompt ambigu ou factuellement erroné et observer la réponse du modèle.

**Exemples de prompt :**
- Décrire « la guerre de 2076 sur Mars »
- Fournir des détails sur une loi imaginaire

L'objectif est d'analyser comment le modèle invente des détails ou admet son ignorance.


In [ ]:
# ============================
# Cellule 7 : Test de Fabrication (Hallucination)
# ============================

prompt_fabrication = """
Tu es journaliste à la fin du 21è siècle, décris-moi la célèbre guerre de 2076 sur la planète Mars :
- Qui étaient les grandes puissances en conflit ?
- Quels traités de paix ont été signés ?
"""

response_fabrication = client.chat.completions.create(
    messages=[{"role": "user", "content": prompt_fabrication}],
    model="gpt-5-mini"
)

print("Réponse du modèle :\n")
print(response_fabrication.choices[0].message.content)


# Responses API : La Nouvelle Génération (2025)

OpenAI a introduit la **Responses API** comme nouvelle approche recommandée pour interagir avec les modèles. Elle offre plusieurs avantages par rapport à Chat Completions :

**Avantages de la Responses API :**
- **Persistance d'état** : Avec `store: true`, les réponses sont sauvegardées et peuvent être chaînées
- **Meilleure utilisation du cache** : 40-80% d'économies sur les tokens répétés
- **Boucle agentique** : Support natif pour les appels d'outils multiples
- **Chaînage simplifié** : `previous_response_id` pour maintenir le contexte

**Chat Completions vs Responses API :**

| Aspect | Chat Completions | Responses API |
|--------|------------------|---------------|
| Syntaxe | `client.chat.completions.create()` | `client.responses.create()` |
| État | Manuel (passer tous les messages) | Automatique avec `store: true` |
| Cache | Basique | Optimisé (40-80% économies) |
| Outils | Support | Support + boucle agentique |

**Note** : Chat Completions reste supporté et fonctionnel. La Responses API est recommandée pour les nouveaux projets.

In [ ]:
# ============================
# Cellule : Exemple Responses API
# ============================

# Exemple avec la Responses API (nouvelle approche recommandée)
# Note: Nécessite openai >= 1.50.0

try:
    # Premier appel avec store=True pour activer la persistance
    response1 = client.responses.create(
        model="gpt-5-mini",
        store=True,
        input="Je m'appelle Alice et j'aime la programmation Python."
    )
    
    print("=== Responses API - Premier appel ===")
    print(f"Response ID: {response1.id}")
    if response1.output:
        print(f"Contenu: {response1.output[0].content[:200]}...")
    
    # Deuxième appel avec chaînage (le contexte est préservé)
    response2 = client.responses.create(
        model="gpt-5-mini",
        store=True,
        previous_response_id=response1.id,
        input="Quel est mon prénom et qu'est-ce que j'aime?"
    )
    
    print("\n=== Responses API - Appel chaîné ===")
    print(f"Response ID: {response2.id}")
    if response2.output:
        print(f"Contenu: {response2.output[0].content}")
    
    print("\n✅ La Responses API fonctionne correctement!")
    
except AttributeError:
    print("⚠️ La Responses API n'est pas disponible dans cette version d'openai.")
    print("   Installez la dernière version: pip install --upgrade openai")
except Exception as e:
    print(f"⚠️ Erreur: {type(e).__name__}: {e}")
    print("   La Responses API peut ne pas être disponible pour tous les comptes.")

# Conclusion de cette Introduction

Nous avons couvert les points suivants :
- Les bases de l'IA générative et des LLMs
- La configuration et l'appel rapide à un modèle via l'API OpenAI
- Les notions de tokenisation et leur impact sur la génération
- Un aperçu des « hallucinations » (fabrications) pouvant survenir dans les réponses du modèle

## Pistes pour Aller Plus Loin

- **Varier la température :** Expérimente avec différentes valeurs (0.0, 0.7, etc.) pour influencer la créativité des réponses.
- **Explorer l'API Chat :** Utilise l'API de chat pour gérer des dialogues contextuels complexes plutôt que l'API `Completion`.
- **Mise en place de la RAG :** Intègre une approche de Retrieval Augmented Generation pour limiter les hallucinations en fournissant des sources documentaires externes.
- **Autres applications :** Essaie la traduction, la génération de code (similaire à Copilot) ou la création de contenu marketing.

**Prochaines étapes dans le cours :**
1. Approfondir le **prompt engineering** (structure des prompts, chaînes d'invocations, etc.).
2. Explorer d'autres types de modèles génératifs (images, audio).
3. Analyser en détail les **enjeux éthiques** (biais, usage responsable, confidentialité).


### Liens utiles et bonnes pratiques

1. [Documentation OpenAI](https://platform.openai.com/docs/) :  
   - Consulter la section *chat completions* pour tous les paramètres disponibles (température, top_p, frequency_penalty, etc.).
2. [Choisir un modèle adapté](/docs/models) :  
   - **gpt-5-mini** pour des réponses rapides et économiques,  
   - **gpt-5** pour des tâches plus complexes nécessitant plus de réflexion.
3. [Exemples officiels](/docs/examples) :  
   - Vous y trouverez des prompts pour différents usages : résumé, traduction, JSON structuré, etc.
4. [Prompt Engineering Guide](/docs/guides/prompt-engineering) :  
   - Conseils avancés pour concevoir des prompts clairs et informatifs.


---

Merci d'avoir suivi cette introduction au monde de l'IA générative !